In [8]:
from blackjacksim.entities import *
from blackjacksim.strategies import basic
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
# import jtplot module in notebook
from jupyterthemes import jtplot
import multiprocessing as mp
from time import time

# set "context" (paper, notebook, talk, poster)
# scale font-size of ticklabels, legend, etc.
# remove spines from x and y axes and make grid dashed
jtplot.style(context='poster', fscale=1.4, spines=False, gridlines='--')

startingPool = 3000

def one_round(procnum, send_end):
    data = []
    house = Blackjack32()
    wallet = PlayerWallet(1,startingPool)
    d = Dealer()
    p = Player(basic, wallet, house)
    s = Shoe(8,.25)
    for t in range(100):
        s = p.deal(s)
        s = d.deal(s)
        s = p.play(s, d.up_card)
        s = d.play(s, p.hands)
        for h in p.hands:
            wallet.take_payout(house.payout(h, d.hand))
        rw, rp = wallet.finish_round()
        data.append({'Round':t,'Pool':wallet.wager_pool, 'Wager':rw, 'Payout':rp, 'Shoe Penetration':s._penetration_state})
    send_end.send(data)


df_list = []
for n in range(1):
    pipe_list = []
    start = time()
    processes = []
    for i in tnrange(100):
        recv_end, send_end = mp.Pipe(False)
        p = mp.Process(target=one_round, args=(i, send_end))
        processes.append(p)
        pipe_list.append(recv_end)
        p.start()

    for process in processes:
        process.join()
    df_list.extend([x.recv() for x in pipe_list])
data = [i for s in df_list for i in s]


In [9]:
print(len(data))

10000


In [5]:
from blackjacksim.entities import *
from blackjacksim.strategies import basic
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from multiprocessing import Process, Manager, Pool

startingPool = 3000
    
data = []
deck = Deck()

def one_round(x):
    house = Blackjack32()
    wallet = PlayerWallet(1,startingPool)
    d = Dealer()
    p = Player(basic, wallet, house)
    s = Shoe(6,.25)
    _card = s.draw(1)[0]
    for t in range(1):
        s = p.deal(s)
        s = d.deal(s)
        s = p.play(s, d.up_card)
        s = d.play(s, p.hands)
        for h in p.hands:
            wallet.take_payout(house.payout(h, d.hand))
        rw, rp = wallet.finish_round()
        data.append({'Round':t,'Pool':wallet.wager_pool, 'Wager':rw, 'Payout':rp, 'Shoe Penetration':s._penetration_state, 'RemovedCard':_card.value})
        
for i in tnrange(100000):
    one_round(i)
            
print(len(data))
df = pd.DataFrame(list(data))
df['Advantage'] = (df.Payout-df.Wager)


100000


In [8]:
print(df.groupby('RemovedCard').mean().Advantage*100)
print(df.groupby('RemovedCard').std().Advantage*100)

RemovedCard
2    -0.141425
3     1.879749
4    -0.945824
5     1.331437
6    -0.621447
7    -1.108358
8    -1.519086
9    -0.932280
10   -0.906026
11    1.076803
Name: Advantage, dtype: float64
RemovedCard
2     116.934311
3     118.188265
4     116.775394
5     117.906368
6     116.158242
7     118.047705
8     117.337043
9     117.360181
10    117.450602
11    118.024076
Name: Advantage, dtype: float64
